# Six-bar mechanism balancing

In [1]:
from BetaShF import *
from BetaShM import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import scipy as sp
import time
from np_differential_evolution import differential_evolution
from cnsg_differential_evolution import cnsg_differential_evolution

assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
np.random.seed(1)

## Util

In [2]:
def cleanData(samples, fitness, forces, moments):
    filterF = forces < 1
    filterM = moments < 1
    f = np.logical_and(filterF, filterM)
    print(f.shape)
    return samples[f], fitness[f], forces[f], moments[f]

In [3]:
def saveData(samples, fitness, forces, moments):
    now = str(int(time.time()))
    np.savetxt(now + "Population.txt", samples)
    np.savetxt(now + "Fitness.txt", fitness)
    np.savetxt(now + "ShForces.txt", forces)
    np.savetxt(now + "ShMoments.txt", moments)
    return now

In [4]:
def logSample(now, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open(name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(now + "Population.txt", s)
    appendToFile(now + "Fitness.txt", str(fitness))
    appendToFile(now + "ShForces.txt", str(force))
    appendToFile(now + "ShMoments.txt", str(moment))

## Problem definition

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

$$
f(x) = \alpha ShF(x) + (1 - \alpha) ShM(x)
$$
$$
\therefore \alpha = \frac{f(x) - ShM(x)}{ShF(x) - ShM(x)}
$$

In [5]:
def objective(s, ShF, ShM, alpha):
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

## Sample generation

In [6]:
#nSamples, nGen = 300, 500 # Tune here
nSamples, nGen = 100, 500 # Tune here

nVariables = 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

### Differential Evolution

#### Define boundaries

In [7]:
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
print(bounds)

[[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


In [8]:
eTimeDE, goodSols = 0, 0
startTime = str(int(time.time()))
print("Saving data with the prefix:", startTime)
for s in range(nSamples):
    alpha = np.random.normal(0.5, 0.20) # To get values near 0.5
    alpha = np.min([1, np.max([0, alpha])]) # Box constraints [0, 1]
    start = time.perf_counter()
    r = differential_evolution(objective, bounds, popSize = 50, nMax = nGen, args = (ShF, ShM, alpha), cR = 0.7)
    #r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM, alpha), maxiter = nGen, popsize = 50, polish = False)
    #r = cnsg_differential_evolution(objective, bounds, args = (ShF, ShM, alpha), MaxGenerations = nGen, popsize = 50)
    end = time.perf_counter()
    sample, fitness, force, moment = r['x'], r["fun"], ShF(r['x']), ShM(r['x'])
    #sample, fitness, force, moment = r.x, r.fun, ShF(r.x), ShM(r.x)
    #sample, fitness, force, moment = r, objective(r, ShF, ShM, alpha), ShF(r), ShM(r)
    if force < 1 and moment < 1:
        goodSols += 1
        logSample(startTime, sample, fitness, force, moment)
    print("Sample ", s, ":\tshF: ", force, "\tshM: ", moment, "\talpha = ", alpha, sep = "")
    eTimeDE += (end - start) #Time in seconds
eTimeDE /= nSamples

Saving data with the prefix: 1676302198
Sample 0:	shF: 0.05821756310742721	shM: 1.4432163818823391	alpha = 0.8248690727326484
Sample 1:	shF: 0.7561766000063185	shM: 0.27418326634003076	alpha = 0.37764871726998495
Sample 2:	shF: 0.7966349391175427	shM: 0.2519307687640336	alpha = 0.32822802883295155
Sample 3:	shF: 0.6708992226309645	shM: 0.33433016861927156	alpha = 0.4412843285609496
Sample 4:	shF: 0.7510117134744076	shM: 0.27732108381450515	alpha = 0.38252178369851314
Sample 5:	shF: 0.44101738424887477	shM: 0.5489622389901959	alpha = 0.5102344571777618
Sample 6:	shF: 0.0829721736436574	shM: 0.9872488571869961	alpha = 0.6363422463752267
Sample 7:	shF: 0.8392301663800743	shM: 0.23429117148485618	alpha = 0.2527040606706909
Sample 8:	shF: 0.18962728279285418	shM: 0.832917500588421	alpha = 0.554632260716062
Sample 9:	shF: 0.38710529242528546	shM: 0.6062560309259302	alpha = 0.5189766376896867
Sample 10:	shF: 0.8244515598877153	shM: 0.2396975124303039	alpha = 0.2834822289476037
Sample 11:	shF:

In [ ]:
print("Average time of execution:", eTimeDE, "seconds. It was run", nSamples, "times.")

In [ ]:
print("Went from ", nSamples, " samples to ", goodSols, " after cleaning data", sep = "")

### Pymoo

In [ ]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=nVariables,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=bounds[:, 0],
                         xu=bounds[:, 1]
        )

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = ShF(x)
        f2 = ShM(x)

        out["F"] = [f1, f2]
        out["G"] = []


problem = MyProblem()

In [ ]:
algorithm = NSGA2(
    pop_size=nSamples, 
    #n_offsprings=nSamples, # Default is none and sets n_offsprings = pop_size
    sampling=FloatRandomSampling(),
    crossover=UniformCrossover(prob=0.75),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
termination = get_termination("n_gen", nGen)

In [ ]:
start = time.perf_counter()
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
end = time.perf_counter() 
eTimeMOO = (end - start)

X = res.X
F = res.F

In [ ]:
print("Time of execution:", eTimeMOO, "seconds.")

In [ ]:
print(X.shape)
print(F.shape)
samples = np.copy(X)
fitness = np.average(F, axis = 1)
shForces = F[:, 0]
shMoments = F[:, 1]

In [ ]:
samples, fitness, shForces, shMoments = cleanData(samples, fitness, shForces, shMoments)
print("Went from ", nSamples, " samples to ", samples.shape[0], " after cleaning data", sep = "")

In [ ]:
prefix = saveData(samples, fitness, shForces, shMoments)
print("Saving data with the prefix:", prefix)